# Traductor de texto japonés a español

In [ ]:
# Install the transformers library from HuggingFace
! pip install transformers torch pytesseract

In [ ]:
# Install the sentencepiece and sacremoses libraries to tokenize the text
! pip install sentencepiece sacremoses

In [4]:
! pip install python-dotenv

In [1]:
# Install deep-translator to translate the text
! pip install deep-translator

# Google translate

In [16]:
from xml.etree import ElementTree as ET
from deep_translator import GoogleTranslator
import os
import html
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import re

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave API de Google Translator desde las variables de entorno
google_api_key = os.getenv("GOOGLE_TRANSLATE_API_KEY")

# Asume que esta función se define en tu entorno con la clave API configurada
google_api_key = google_api_key  # Asegúrate de manejar tus claves API de manera segura
traductor = GoogleTranslator(api_key=google_api_key, source='ja', target='es')

def eliminar_explicaciones(texto):
    # Eliminar texto dentro de paréntesis, incluyendo los paréntesis mismos
    texto_sin_explicaciones = re.sub(r'\([^)]*\)', '', texto)
    # Agregar más reglas de limpieza aquí si es necesario
    return texto_sin_explicaciones.strip()

def traducir_texto(texto):
    if texto:
        traduccion = traductor.translate(texto)
        traduccion_limpia = html.unescape(traduccion)
        traduccion_final = eliminar_explicaciones(traduccion_limpia)
        return traduccion_final
    return ""

def traducir_archivo_xml_concurrente(xml_path, output_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    textos_para_traducir = [(elem, elem.text.strip()) for elem in root.findall('.//text') if elem.text and elem.text.strip()]
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        for elem, traduccion in zip(textos_para_traducir, executor.map(lambda x: traducir_texto(x[1]), textos_para_traducir)):
            elem[0].set('translated_text', traduccion)
    
    file_name = os.path.splitext(os.path.basename(xml_path))[0] + "-es(GT).xml"
    output_file_path = os.path.join(output_folder, file_name)
    tree.write(output_file_path, encoding="utf-8", xml_declaration=True)
    print(f"Archivo traducido guardado en: {output_file_path}")

xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
output_folder = '/home/carol/code/cparran/MOJI/raw_data/output'
traducir_archivo_xml_concurrente(xml_input_path, output_folder)

Archivo traducido guardado en: /home/carol/code/cparran/MOJI/raw_data/output/ARMS_lite-es(GT).xml


Llama a la función definida en "traductor_xml.py", esta usa Google Translator

In [25]:
import sys
sys.path.append('/home/carol/code/cparran/MOJI/')
from moji.traductor_xml import traducir_archivo_xml
xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
traducir_archivo_xml(xml_input_path)            # aloja el archivo traducido en la carpeta donde se aloja la notebook, se puede modificar.

Archivo traducido guardado como: ARMS_lite-es.xml


# Deepl Translator

Es más confiable

# DeepL Translator (más rápido)

In [14]:
from xml.etree import ElementTree as ET
from deep_translator import DeeplTranslator
import os
import html
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor
import re

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave API de DeepL desde las variables de entorno
api_key = os.getenv("DEEPL_API_KEY")

# Asume que esta función se define en tu entorno con la clave API configurada
api_key = api_key  # Asegúrate de manejar tus claves API de manera segura
traductor = DeeplTranslator(api_key=api_key, source="ja", target="es", use_free_api=True)

def eliminar_explicaciones(texto):
    # Eliminar texto dentro de paréntesis, incluyendo los paréntesis mismos
    texto_sin_explicaciones = re.sub(r'\([^)]*\)', '', texto)
    # Agregar más reglas de limpieza aquí si es necesario
    return texto_sin_explicaciones.strip()

def traducir_texto(texto):
    if texto:
        traduccion = traductor.translate(texto)
        traduccion_limpia = html.unescape(traduccion)
        traduccion_final = eliminar_explicaciones(traduccion_limpia)
        return traduccion_final
    return ""

def traducir_archivo_xml_concurrente(xml_path, output_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    textos_para_traducir = [(elem, elem.text.strip()) for elem in root.findall('.//text') if elem.text and elem.text.strip()]
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        for elem, traduccion in zip(textos_para_traducir, executor.map(lambda x: traducir_texto(x[1]), textos_para_traducir)):
            elem[0].set('translated_text', traduccion)
    
    file_name = os.path.splitext(os.path.basename(xml_path))[0] + "-es(DL).xml"
    output_file_path = os.path.join(output_folder, file_name)
    tree.write(output_file_path, encoding="utf-8", xml_declaration=True)
    print(f"Archivo traducido guardado en: {output_file_path}")

xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
output_folder = '/home/carol/code/cparran/MOJI/raw_data/output'
traducir_archivo_xml_concurrente(xml_input_path, output_folder)


Archivo traducido guardado en: /home/carol/code/cparran/MOJI/raw_data/output/ARMS_lite-es(DL).xml


# OPEN AI Translator

In [59]:
#! pip install openai==0.27.0
! pip install deep-translator==1.9.0

  Attempting uninstall: deep-translator
    Found existing installation: deep-translator 1.11.4
    Uninstalling deep-translator-1.11.4:
      Successfully uninstalled deep-translator-1.11.4


In [26]:
import openai
import os
from dotenv import load_dotenv
import html
from concurrent.futures import ThreadPoolExecutor
import re
from xml.etree import ElementTree as ET

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener la clave API de OpenAI desde las variables de entorno
openai_api_key = os.getenv("OPENAI_API_KEY")

# Configurar la API de OpenAI
openai.api_key = openai_api_key

def eliminar_explicaciones(texto):
    # Eliminar texto dentro de paréntesis, incluyendo los paréntesis mismos
    texto_sin_explicaciones = re.sub(r'\([^)]*\)', '', texto)
    return texto_sin_explicaciones.strip()

def traducir_texto(texto):
    if not texto:
        return ""

    # Traducción con GPT-3.5 Turbo (modelo de chat)
    traduccion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f" Traduce el siguiente diálogo de japonés a español. Evita proporcionar definiciones o explicaciones innecesarias. Concéntrate en ofrecer traducciones precisas y naturales :\n\n{texto}"}
        ]
    )['choices'][0]['message']['content']
    
    traduccion_limpia = html.unescape(traduccion)
    
    return eliminar_explicaciones(traduccion_limpia)

def traducir_archivo_xml_concurrente(xml_path, output_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    textos_para_traducir = [(elem, elem.text.strip()) for elem in root.findall('.//text') if elem.text and elem.text.strip()]
    
    with ThreadPoolExecutor(max_workers=20) as executor:
        for elem, traduccion in zip(textos_para_traducir, executor.map(lambda x: traducir_texto(x[1]), textos_para_traducir)):
            elem[0].set('translated_text', traduccion)
    
    file_name = os.path.splitext(os.path.basename(xml_path))[0] + "-es(GPT).xml"
    output_file_path = os.path.join(output_folder, file_name)
    tree.write(output_file_path, encoding="utf-8", xml_declaration=True)
    print(f"Archivo traducido guardado en: {output_file_path}")

xml_input_path = '/home/carol/code/cparran/MOJI/raw_data/ARMS/ARMS_lite.xml'
output_folder = '/home/carol/code/cparran/MOJI/raw_data/output'
traducir_archivo_xml_concurrente(xml_input_path, output_folder)


Archivo traducido guardado en: /home/carol/code/cparran/MOJI/raw_data/output/ARMS_lite-es(GPT).xml
